In [1]:
import torch.nn as nn
import torch.nn.functional as F
import math

class Linear(nn.Module):

    def __init__(self, num_classes=2):

        super(Linear, self).__init__()

        self.number_class   = num_classes

        _size_image     = 100* 100
        _num1           = 50
        _num2           = 50
        
        self.fc1        = nn.Linear(_size_image, _num1, bias=True)
        self.fc2        = nn.Linear(_num1, _num2, bias=True)
        self.fc3        = nn.Linear(_num2, num_classes, bias=True)

        self.fc_layer1  = nn.Sequential(self.fc1, nn.ReLU(True))
        self.fc_layer2  = nn.Sequential(self.fc2, nn.ReLU(True))
        self.fc_layer3  = nn.Sequential(self.fc3, nn.ReLU(True))
        
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.kaiming_normal_(self.fc2.weight)
        nn.init.kaiming_normal_(self.fc3.weight)
        
        self.fc1.bias.data.zero_()
        self.fc2.bias.data.zero_()
        self.fc3.bias.data.zero_()
        
        self.classifier = nn.Sequential(self.fc_layer1, self.fc_layer2, self.fc_layer3)
        #self._initialize_weight()        
        
    def _initialize_weight(self):
        #Linear = m.__class__.__name__
        for m in self.modules():
            
            n = m.in_features
            
            m.weight.data.uniform_(- 1.0 / math.sqrt(n), 1.0 / math.sqrt(n))

            if m.bias is not None:

                m.bias.data.zero_()

    def forward(self, x):

        x = x.view(x.size(0), -1)
        x = self.classifier(x)

        return x


In [ ]:
# -----------------------------------------------------------------------------
# import packages
# -----------------------------------------------------------------------------
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import argparse
import sys
import os
import numpy as np
import time
import datetime 
import csv
import configparser
import argparse
import platform

from torchvision import datasets, transforms
from torch.autograd import Variable
from random import shuffle

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision
import os

# -----------------------------------------------------------------------------
# load dataset
# -----------------------------------------------------------------------------

batch_size = 10
transform = transforms.Compose([#transforms.Resize((256,256)),  
                                transforms.Grayscale(),		
# the code transforms.Graysclae() is for changing the size [3,100,100] to [1, 100, 100] (notice : [channel, height, width] )
                                transforms.ToTensor(),])
train_data_path = 'horse-or-human/horse-or-human/train'
set_train   =   torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
loader_train = torch.utils.data.DataLoader(set_train, batch_size=batch_size, shuffle=True, num_workers=0)  


validation_data_path = 'horse-or-human/horse-or-human/validation'
set_test   =   torchvision.datasets.ImageFolder(root=validation_data_path, transform=transform)
loader_test = torch.utils.data.DataLoader(set_test, batch_size=batch_size, shuffle=False, num_workers=0)


num_classes = 2

# -----------------------------------------------------------------------------
# load neural network model
# -----------------------------------------------------------------------------

#from Linear import *
model = Linear(num_classes=num_classes)
#model.apply(_initialize_weight)
# -----------------------------------------------------------------------------
# Set the flag for using cuda
# -----------------------------------------------------------------------------

bCuda = 0

if bCuda:
 
    model.cuda()

# -----------------------------------------------------------------------------
# optimization algorithm
# -----------------------------------------------------------------------------

learning_rate = 1e-4
optimizer   = optim.SGD(model.parameters(), lr = learning_rate, weight_decay=1e-4)
objective   = nn.CrossEntropyLoss()

# -----------------------------------------------------------------------------
# function for training the model
# -----------------------------------------------------------------------------

def train():

    # print('train the model at given epoch')

    loss_train          = []
    correct         = 0

    model.train()

    for idx_batch, (data, target) in enumerate(loader_train):

        if bCuda:
        
            data, target    = data.cuda(), target.cuda()

        data, target    = Variable(data), Variable(target)

        optimizer.zero_grad()

        output  = model(data)
        loss    = objective(output, target)

        loss.backward()
        optimizer.step()

        loss_train_batch    = loss.item() / len(data)
        loss_train.append(loss_train_batch)
        
        pred        = output.data.max(1)[1]
        correct     += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    loss_train_mean     = np.mean(loss_train)
    loss_train_std      = np.std(loss_train)
    accuracy_train   = 100. * float(correct) / len(loader_train.dataset)
    return {'loss_train_mean': loss_train_mean, 'loss_train_std': loss_train_std, 'accuracy_train': accuracy_train }

# -----------------------------------------------------------------------------
# function for testing the model
# -----------------------------------------------------------------------------

def test():

    # print('test the model at given epoch')

    accuracy_test   = []
    loss_test       = 0
    correct         = 0

    model.eval()

    for idx_batch, (data, target) in enumerate(loader_test):

        if bCuda:
        
            data, target    = data.cuda(), target.cuda()

        data, target    = Variable(data), Variable(target)

        output  = model(data)
        loss    = objective(output, target)

        loss_test   += loss.item()
        pred        = output.data.max(1)[1]
        correct     += pred.eq(target.data.view_as(pred)).cpu().sum()

    loss_test       = loss_test / len(loader_test.dataset)
    accuracy_test   = 100. * float(correct) / len(loader_test.dataset)

    return {'loss_test': loss_test, 'accuracy_test': accuracy_test}

# -----------------------------------------------------------------------------
# iteration for the epoch
# -----------------------------------------------------------------------------

epoch = 6000

loss_train_mean = []
loss_train_std = []
loss_test = []
accuracy_test = []
accuracy_train = []

for e in range(epoch):
        
    result_train    = train()
    result_test     = test()
    
    print(e)
    print(result_train)
    print(result_test)
    
    loss_train_mean.append(result_train['loss_train_mean'])
    loss_train_std.append(result_train['loss_train_std'])
    loss_test.append(result_test['loss_test'])
    accuracy_test.append(result_test['accuracy_test'])
    accuracy_train.append(result_train['accuracy_train'])
    #loss_train_mean[e]  = result_train['loss_train_mean']
    #loss_train_std[e]   = result_train['loss_train_std']
    #loss_test[e]        = result_test['loss_test']
    #accuracy_test[e]    = result_test['accuracy_test']
    


0
{'loss_train_mean': 0.07024134039548167, 'loss_train_std': 0.006351471050826842, 'accuracy_train': 55.98831548198637}
{'loss_test': 0.06885433802381158, 'accuracy_test': 52.34375}
1
{'loss_train_mean': 0.06733803706923074, 'loss_train_std': 0.004141584824262507, 'accuracy_train': 62.804284323271666}
{'loss_test': 0.06910228740889579, 'accuracy_test': 50.78125}
2
{'loss_train_mean': 0.0660668504469603, 'loss_train_std': 0.0047103597258013755, 'accuracy_train': 63.87536514118793}
{'loss_test': 0.05778232298325747, 'accuracy_test': 78.125}
3
{'loss_train_mean': 0.06188346541456972, 'loss_train_std': 0.006493312151659639, 'accuracy_train': 69.32814021421616}
{'loss_test': 0.051716232439503074, 'accuracy_test': 81.25}
4
{'loss_train_mean': 0.05867989764712886, 'loss_train_std': 0.007125917710585718, 'accuracy_train': 73.51509250243427}
{'loss_test': 0.04810676095075905, 'accuracy_test': 84.765625}
5
{'loss_train_mean': 0.05669036688652515, 'loss_train_std': 0.007152097112795123, 'accuracy

{'loss_test': 0.030377010501979385, 'accuracy_test': 84.765625}
45
{'loss_train_mean': 0.0331717085088093, 'loss_train_std': 0.011079446896514663, 'accuracy_train': 87.5365141187926}
{'loss_test': 0.030206849980459083, 'accuracy_test': 85.546875}
46
{'loss_train_mean': 0.032807848713103674, 'loss_train_std': 0.010224727212247533, 'accuracy_train': 88.1207400194742}
{'loss_test': 0.02826544597701286, 'accuracy_test': 89.0625}
47
{'loss_train_mean': 0.032555092922857506, 'loss_train_std': 0.010588404576723106, 'accuracy_train': 88.41285296981499}
{'loss_test': 0.0286677696640254, 'accuracy_test': 87.109375}
48
{'loss_train_mean': 0.03247886822764321, 'loss_train_std': 0.01087061459254923, 'accuracy_train': 88.51022395326193}
{'loss_test': 0.03221242988365702, 'accuracy_test': 83.984375}
49
{'loss_train_mean': 0.03211459348006652, 'loss_train_std': 0.009307521251647737, 'accuracy_train': 88.1207400194742}
{'loss_test': 0.03397563557518879, 'accuracy_test': 82.421875}
50
{'loss_train_mean'

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(loss_train_mean,label = "training loss")
plt.plot(loss_test,label="test loss")
plt.xlabel('iteration')
plt.ylabel('loss') 
plt.title('Loss')
plt.legend() 
plt.show() 

plt.plot(accuracy_train,label="train accuracy")
plt.plot(accuracy_test,label="test accuracy")
plt.xlabel('iteration')
plt.title('Accuracy')
plt.legend()
plt.show()

In [ ]:
print("\t\ttraining\t\ttest\t\t")
print("=================================================================================")
print("loss\t\t"+"{0:.10f}".format(loss_train_mean[-1])+"\t\t"+"{0:.10f}".format(loss_test[-1])+"\t\t")
print("=================================================================================")
print("accuracy\t"+"{0:.10f}".format(accuracy_train[-1])+"\t\t"+ "{0:.10f}".format(accuracy_test[-1])+"\t\t")